In [1]:
import time
import pickle
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

In [2]:
# fix professors
from owlready2 import *
main_onto_name = 'main-ResearchMentorOntology.owl'
onto_path.append("/Users/lauramiron/Desktop/CS270/project")
onto = get_ontology(main_onto_name)
vivoNS = onto.get_namespace("http://vivoweb.org/ontology/core")
onto.load()

for i in vivoNS.FacultyMember.instances():
    for RA in i.hasResearchArea:
        i.hasResearchArea.remove(RA)
onto.save()

In [7]:
import pdb
med_students_base_url = 'https://med.stanford.edu/profiles/browse?p={0}&affiliations=capMdStudent&ps=100'

driver = webdriver.Chrome()
med_student_dict = {}
print('scraping med students...')
for pg_num in range(1,4):
    print('downloading page '+str(pg_num))
    url = med_students_base_url.format(pg_num)
    driver.get(url)
    time.sleep(8)
    profiles = driver.find_elements_by_class_name("mini-profile")
    for pfl in profiles:
        media_heading_elem = pfl.find_element_by_class_name("media-heading")
        url_name = media_heading_elem.get_attribute("href").split('/')[-1]
        name = media_heading_elem.find_element_by_tag_name('h4').get_attribute('innerText')
        first_name = name.split(' ')[0].replace(' ','')
        last_name = name.split(' ')[-1].replace(' ','')
        program_info = media_heading_elem.find_element_by_tag_name('h5').get_attribute('innerText')
        program = program_info.split(' ')[0]
        grad_year = program_info.split(' ')[-1]
        med_student_dict[url_name] = {'firstName':first_name, 'lastName': last_name, 'program': program, 'gradDate': grad_year}
driver.quit()
med_student_dict_name = 'med_student_simple.p'
print('saving to '+med_student_dict_name)        
with open(med_student_dict_name,'wb+') as f:
    pickle.dump(med_student_dict,f)

scraping med students...
downloading page 1
downloading page 2
downloading page 3
saving to med_student_simple.p


In [2]:
from owlready2 import *

main_onto_name = 'main-ResearchMentorOntology.owl'
onto_path.append("/Users/lauramiron/Desktop/CS270/project")
onto = get_ontology(main_onto_name)
vivoNS = onto.get_namespace("http://vivoweb.org/ontology/core")
onto.load()

get_ontology("http://www.semanticweb.org/lauramiron/ontologies/2019/1/merged-research-mentor#")

In [14]:
med_student_profile_url = 'https://med.stanford.edu/profiles/'
main_onto_name = 'main-ResearchMentorOntology.owl'
med_student_dict_name = 'med_student_simple.p'
med_student_dict = pickle.load(open(med_student_dict_name,'rb'))

for url_name, values in med_student_dict.items():
    first_name = values['firstName']
    last_name = values['lastName']
    program = values['program']
    grad_year = values['gradDate']
    
    med_indiv = onto.MedicalStudent((first_name+last_name).replace(' ',''))
    med_indiv.firstName = first_name
    med_indiv.lastName = last_name
    med_indiv.graduationDateExp.append(grad_year)
    med_indiv.uri = url_name
    med_indiv.hasProfilePage.append(med_student_profile_url+url_name)

onto.save()

In [10]:
import pandas as pd
import csv

def is_sub_org(sub_org):
    for pfx in org_prefixes:
        if sub_org.startswith(pfx):
            return True
    return False
    
def add_sub_org(onto,sub_org):
    if sub_org.startswith('Departmentof'):
        sub_org = 'Stanford'+sub_org.strip('Departmentof')+'Department'
        if len(onto.search(iri='*'+sub_org)) == 0:
            sub_org_indiv = vivoNS.AcademicDepartment(sub_org)
            sub_org_indiv.subOrganizationWithin.append(school_indiv)
    elif sub_org.startswith('Divisionof'):
        sub_org = 'Stanford'+sub_org
        if len(onto.search(iri='*'+sub_org)) == 0:
            sub_org_indiv = vivoNS.Division(sub_org)
            sub_org_indiv.subOrganizationWithin.append(school_indiv)
    elif sub_org.startswith('Centerfor'):
        sub_org = 'Stanford' + sub_org
        if len(onto.search(iri='*'+sub_org)) == 0:
            sub_org_indiv = vivoNS.Center(sub_org)
            sub_org_indiv.subOrganizationWithin.append(school_indiv) 

def add_all_sub_orgs(onto,sub_org):
    print(sub_org)
    sub_sub_orgs = sub_org.split('and')
    first_com = sub_sub_orgs[0]
    if len(sub_sub_orgs) == 1:
        return add_sub_org(onto,first_com)
    for i in range(1,len(sub_sub_orgs)):
        if (is_sub_org(sub_sub_orgs[i])):
            add_sub_org(onto,first_com)
            add_all_sub_orgs(onto,sub_org[len(first_com)+3:])
            break
        else:
            first_com += 'and'
            first_com += sub_sub_orgs[i]
    add_sub_org(onto,first_com)
            
# add MESH terms and PubMed articles
org_prefixes = ['Departmentof','Divisionof','Centerof']
df = pd.read_csv('authors_cleaned.csv',sep=',',skipinitialspace=True,quoting=csv.QUOTE_ALL,engine='python')
for aff in df.afft.unique():
    if not 'Stanford University' in aff: continue
    locs = aff.split(',')
    if len(locs) < 1: continue
    if len(locs) == 1: 
        school = locs[0].strip()
        continue
    else:
        sub_org = locs[0].strip()
        school = locs[1].strip()
    if school == 'Stanford University':
        school_indiv = onto.search(iri='*StanfordUniversity')[0]
    elif school in ('School of Medicine at Stanford University','Stanford University School of Medicine'):
        school_indiv = onto.search(iri='*StanfordUniversitySchoolofMedicine')[0]
    elif school in ('Stanford University Medical Center'):
        school_indiv = onto.search(iri='*StanfordUniversityMedicalCenter')[0]
    else: continue #not affiliated with Stanford
        
    sub_org = sub_org.split('(')[0].split('-')[0].replace(' ','').strip('1]').replace('&','and').replace('/','and').split(';')[0]
    add_all_sub_orgs(onto,sub_org)
    
onto.save()

DepartmentofHealthResearchandPolicy
DepartmentofGenetics
Meta
StanfordUniversity
SchoolofMedicineatStanfordUniversity
DepartmentofSurgery
StanfordUniversitySchoolofMedicine
StanfordUniversity
Meta
Meta
Meta
DepartmentofOphthalmology
DepartmentofMedicine
SchoolofMedicine
PediatricRheumatology
StanfordUniversity
DepartmentofGeologicalSciences
CenterforSleepScienceandMedicine
CenterforSleepScienceandMedicine
DepartmentofBioengineering
HopkinsMarineStation
DepartmentofPsychiatryandBehavioralSciences
DepartmentofPediatrics
DepartmentofMedicine
2DepartmentofPsychiatryandBehavioralSciences
DepartmentofBiology
DepartmentofGeologicalSciences
Psychologydepartment
DepartmentofBiologicalSciences
DivisionofPlasticandReconstructiveSurgery
DepartmentofPhysics
BiophysicsProgram
BiophysicsProgram
DepartmentofPsychiatryandBehavioralSciences
StanfordMedicineWellMDCenter
DepartmentofPsychiatryandBehavioralSciences
DepartmentofElectricEngineering
DepartmentsofRadiologyandElectricEngineering
DepartmentofOrt

StanfordPreventionResearchCenter
DepartmentofMedicine
DivisionofFacialPlasticandReconstructiveSurgery
DivisionofFacialPlasticandReconstructiveSurgery
DepartmentofOtolaryngology
DepartmentofOtolaryngology
DepartmentofCivilandEnvironmentalEngineering
DepartmentofCivilandEnvironmentalEngineering
DivisionofInterventionalRadiology
DivisionofInterventionalRadiology
DepartmentofOrthopaedicSurgery
DepartmentofOrthopaedicSurgery
DepartmentofBiochemistry
DepartmentofStatistics
DepartmentofGenetics
DepartmentofStatistics
CardiothoracicSurgery
DepartmentofMedicine
DepartmentofPsychiatryandBehavioralSciences
DepartmentofBiology
WoodsInstitutefortheEnvironment
DepartmentofGenetics
DepartmentofDermatology
DepartmentofDermatology
DepartmentofPathology
DepartmentofDermatology
DivisionofImmunologyandRheumatology
DepartmentofNeurologicalSurgery
StanfordUniversity
DepartmentofBiology
DivisionofCardiovascularMedicine
3DepartmentofCardiothoracicSurgery
DepartmentofRadiology
DepartmentofStructuralBiology
Dep

QuantitativeSciencesUnit
DepartmentofMedicine
DepartmentofDermatology
CenterforOceanSolutions
DepartmentofBiology
DepartmentofBioengineering
HansenExperimentalPhysicsLaboratory
DepartmentofElectricalEngineering
HansenExperimentalPhysicsLaboratory
DepartmentofMicrobiologyandImmunology
DepartmentofPediatrics
DivisionofGastroenterologyandHepatology
StanfordUniversitySchoolofMedicine
DepartmentofBiology
DepartmentofBiology
DepartmentofRadiology
DepartmentofSurgery
DepartmentofComputerScience
EmmettInterdisciplinaryPrograminEnvironmentandResources
DepartmentofNeurologicalSurgery
DepartmentofClinicalDermatology
dDivisionofHematology
Dept.ofPsychiatryandBehavioralSciences
DepartmentofRadiationOncology
DepartmentofNeurosurgery
DepartmentofPathology
DepartmentofOrthopaedicSurgery
DepartmentofPsychology
aDepartmentofPsychiatryandBehavioralSciences
DepartmentofSurgery
DepartmentofGeneralSurgery
DepartmentofMicrobiologyandImmunology
DepartmentofMicrobiologyandImmunology
DepartmentofBiology
Departm

StanfordCardiovascularInstitute
DepartmentofHealthResearchandPolicy
DepartmentofComparativeMedicine
StanfordPreventionResearchCenter
DepartmentofRadiationOncology
DepartmentofRadiationOncology
DepartmentofMedicine
DepartmentofCivilandEnvironmentalEngineering
DepartmentofCivilandEnvironmentalEngineering
DepartmentofPathology
DepartmentofPathology
StanfordUniversity
DivisionofClinicalMedicineandPopulationHealth
DepartmentofCardiothoracicSurgery
DepartmentofRadiology
StanfordPreventionCenter
DivisionofVascularSurgery
DepartmentofCardiothoracicSurgery
DepartmentofUrology
DepartmentofHealthResearchandPolicy
DepartmentofPediatrics
DepartmentofCardiothoracicSurgery
DivisionofPediatricCardiology
DepartmentOtolaryngology
DepartmentofMedicine
DepartmentofSurgery
DivisionofCardiovascularMedicine
DivisionofCardiovascularMedicine
DivisionofCardiovascularMedicine
DivisionofCardiovascularMedicine
DepartmentofOrthopaedicSurgery
DepartmentofPsychology
StanfordResearchComputingCenterandSchoolofMedicine


DivisionofInfectiousDiseasesandGeographicMedicine
SectionofPediatricCardiology
DepartmentofChemicalandSystemsBiology
DepartmentofChemicalandSystemsBiology
DepartmentofGenetics
DepartmentofChemicalandSystemsBiology
StanfordGenomeTechnologyCenter
DivisionofRheumatology
DepartmentofDermatology
DivisionofGastroenterologyandHepatology
DivisionofNephrology
DepartmentofBioengineering
BiomedicalInformaticsTrainingProgram
DepartmentofChemicalandSystemsBiology
DivisionofOncology
QuantitativeSciencesUnit
DepartmentofMedicine
InstituteforStemCellBiologyandRegenerativeMedicine
DepartmentofOrthopaedicSurgery
DepartmentsofEmergencyMedicine
DepartmentofPsychiatryandBehavioralSciences
DepartmentofPsychiatryandBehavioralSciences
DepartmentofNeurologicalSurgery
0DepartmentofHealthResearchandPolicy
FromtheDepartmentofPsychiatryandBehavioralSciences
dDepartmentofMedicine
eDivisionofGastroenterologyandHepatology
DepartmentofOtolaryngology
DepartmentofRadiationOncology
DepartmentofOtolaryngology
Quantitative

...

DepartmentofOrthopaedicSurgery
DepartmentofNeurologyandNeurosciences
DepartmentofPathologyandMedicine
StanfordHospitalandClinics
DepartmentofPsychiatryandBehavioralSciences
InterventionalCardiology
DepartmentofInterventionalRadiology
DepartmentofPsychiatry
DivisionofPlasticandReconstructiveSurgery
DepartmentofCardiothoracicSurgery
DepartmentofBioengineering
CenterforPrimaryCareandOutcomesResearchandtheDivisionofEmergencyMedicine
DepartmentofNeurosurgeryandStanfordStrokeCenter
DepartmentofAnesthesia
DivisionofImmunologyandRheumatology
DepartmentofRadiology
DepartmentofPediatrics
DepartmentofNeurology
DepartmentofInternalMedicine
DivisionofEmergencyMedicine
DepartmentofNeurosurgery
DepartmentofComparativeMedicine
DepartmentofNeurology
SchoolofMedicine
DepartmentofPediatrics
StanfordPreventionResearchCenter
DepartmentofPathology
DepartmentofSurgery
DepartmentofSurgery
DivisionofPlasticSurgery
CenterforComparativeStudiesinRaceandEthnicity
StanfordUniversity
StanfordCenterforBiomedicalEthic

DivisionofGastroenterologyandHepatology
DivisionofGastroenterologyandHepatology
StanfordUniversityMedicalCenter
DepartmentofSurgery
DepartmentofSurgery
DepartmentofRadiology
DepartmentofInterventionalRadiology
DepartmentofInterventionalRadiology
DepartmentofNeurology
DepartmentofMedicine
DivisionofGastroenterologyandHepatology
DepartmentsofPathologyandPediatrics
StanfordUniversityMedicalCenter
DepartmentofUrology
DepartmentofUrologyandDermatology
DepartmentofSurgery
DepartmentofDermatology
DepartmentofDermatology
bDivisionofGastroenterologyandHepatology
SectionofBariatricandMinimallyInvasiveSurgery
SectionofBariatricandMinimallyInvasiveSurgery
DivisionofNeonatalandDevelopmentalMedicine
FromtheDepartmentofPsychiatryandBehavioralSciences
FromtheDepartmentofPsychiatryandBehavioralSciences
DepartmentofPediatrics
DivisionofGastroenterologyandHepatology
CenterforHealthPolicyandPrimaryCareandOutcomesResearch
DepartmentofOrthopedicSurgery
DepartmentofOrthopedicSurgery
DepartmentofCardiothoraci

DepartmentofUrology
StanfordUniversity
RobertA.ChaseCenterforHandandUpperLimbSurgery
*DepartmentofOrthopaedicSurgery
DepartmentofRadiology
StanfordUniversitySchoolofMedicine
DivisionofCardiothoracicSurgery
DepartmentofRadiationOncology
DepartmentofMedicine
DepartmentofOrthopaedicSurgery
StanfordUniversitySchoolofMedicine
StanfordUniversity
StanfordUniversity
DepartmentofPsychology
DepartmentofPsychology
DepartmentofGenetics
*DepartmentofAnesthesia†DepartmentofOtolaryngology
DepartmentofSurgery
CenterforHealthPolicyandCenterforPrimaryCareandOutcomesResearch
DepartmentofPsychiatryandBehavioralSciences
DepartmentofMaterialScienceandEngineering
StanfordUniversityHospital
DepartmentofRadiology
DepartmentofPathology
DepartmentofBioengineering
StanfordUniversitySchoolofMedicine
DivisionofAbdominalTransplantation
DepartmentofPsychiatryandBehavioralSciences
Dept.ofPsychiatryandBehavioralSciences
DepartmentofCardiothoracicSurgery
DepartmentofOrthopaedicSurgery
DeptofPathology
FromtheDivisionofPl

DepartmentofCardiothoracicSurgery
DepartmentofGynandOb
DepartmentofPsychiatryandBehavioralSciences
DivisionofPlasticandReconstructiveSurgery
DepartmentofMedicine
DivisionofCardiovascularandInterventionalRadiology
StanfordMedicalInformatics
DepartmentsofRadiologyandNeurosurgery
DepartmentsofNeurosurgery
DepartmentofNeurologyandNeurologicalSciences
DivisionofPediatricCardiology
DepartmentofPsychiatryandBehavioralSciences
DivisionofGeneralPediatrics
StanfordUniversity
DepartmentsofGeneticsandMedicine
FertilityPhysiciansofNorthernCalifornia
StanfordCenterforResearchinDiseasePrevention
RadiationOncologyDepartment
DivisionofCardiovascularandInterventionalRadiology
DivisionofOtolaryngology
StanfordUniversityHospital
DivisionofInfectiousDiseasesandAIDSResearch
StanfordMedicalInformatics
StanfordMedicalInformatics
DepartmentofSurgery
MedicalInformatics
DivisionofPulmonaryandCriticalCareMedicine
StanfordUniversitySchoolofMedicine
DepartmentofNeonatalandDevelopmentalMedicine
DepartmentofPsychiatr

DepartmentofPsychology
SUMMIT
StanfordHospitalandClinics
LucillePackardChildren'sHospital
DepartmentofBiologicalSciences
StanfordUniversityMedicalCenter
DivisionofOtolaryngology
PsychiatryandBehavioralSciences
LucilePackardChildren'sHospital
GraduateSchoolofBusiness
DepartmentofDermatology
DepartmentofPsychology
DepartmentofChemistry
DivisionofSportsMedicine
DepartmentofRadiationOncology
DepartmentofBiochemistry
OrthopaedicResearchLaboratory
DepartmentofMicrobiologyandImmunology
SectiononMedicalInformatics
DepartmentofRadiationOncology
DivisionofClinicalPharmacology
DepartmentofAnthropology
DepartmentofPsychiatryandBehavioralScience
SectiononMedicalInformatics
DivisionofGeneralInternalMedicine
SectiononMedicalInformatics
StanfordMagneticResonanceLaboratory
DepartmentofElectricalEngineering
DivisionofOtolaryngology
CenterforBiomedicalEthics
DivisionofGeneralInternalMedicine
DepartmentofPlasticSurgery
DepartmentofPsychiatryandBehavioralScience
DivisionofOrthopaedicSurgery
DivisionofCardi

In [ ]:
def is_sub_org(sub_org):
    for pfx in org_prefixes:
        if sub_org.startswith(pfx):
            return True
    return False
    
def get_sub_org(onto,sub_org,orgs_list):
    if sub_org.startswith('Departmentof'):
        sub_org = 'Stanford'+sub_org.strip('Departmentof')+'Department'
    elif sub_org.startswith('Divisionof'):
        sub_org = 'Stanford'+sub_org
    elif sub_org.startswith('Centerfor'):
        sub_org = 'Stanford' + sub_org
    else return
    indiv = onto.search(iri='*'+sub_org)
    orgs_list.append(indiv)
                            
def get_all_sub_orgs(onto,sub_org,orgs_list):
    sub_sub_orgs = sub_org.split('and')
    first_com = sub_sub_orgs[0]
    if len(sub_sub_orgs) == 1:
        return add_sub_org(onto,first_com,orgs_list)
    for i in range(1,len(sub_sub_orgs)):
        if (is_sub_org(sub_sub_orgs[i])):
            add_sub_org(onto,first_com,orgs_list)
            add_all_sub_orgs(onto,sub_org[len(first_com)+3:],orgs_list)
            break
        else:
            first_com += 'and'
            first_com += sub_sub_orgs[i]
    add_sub_org(onto,first_com,orgs_list)


df = pd.read_csv('authors_cleaned.csv',sep=',',skipinitialspace=True,quoting=csv.QUOTE_ALL,engine='python')
for line in df:
    author = df['author']
    pmid = df['pmid']
    afft = df['afft']
    first_name = author.split(' ')[0]
    last_name = author.split(' ')[1]
    faculty_indiv = onto.search(iri='*'+first_name+last_name)
    if not faculty_indiv: faculty_indiv = onto.search(iri='*'+author.replace(' ',''))
    if not faculty_indiv: continue
    
    sub_orgs = []
    sub_org_str = afft.split('(')[0].split('-')[0].replace(' ','').strip('1]').replace('&','and').replace('/','and').split(';')[0]
    get_all_sub_orgs(onto,sub_org_str,sub_orgs)
    for sub_org in sub_orgs:
        if not sub_org in faculty_indiv.currentMemberOf:
            faculty_indiv.currentMemberOf.append(sub_org)
        article_name = 'PubMedArticle'+pmid
        article_indiv = onto.search(iri='*'+article_name)
        if not article_indiv:
            article_indiv = vivoNS.AcademicArticle(article_name)
            article_indiv.pmid = pmid
        if article_indiv not in faculty_indiv.authorOf:
            faculty_indiv.authorOf.append(article_indiv)














